In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Music recommender system


### Content-based filters



### Importing required libraries

First, we'll import all the required libraries.

In [2]:
import numpy as np
import pandas as pd

In [3]:
from typing import List, Dict

We are going to use TfidfVectorizer from the Scikit-learn package again.

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

### Dataset


This dataset contains name, artist, and lyrics for *57650 songs in English*.

In [5]:
%cd /content/drive/MyDrive/ML_FSP
songs = pd.read_csv('songdata.csv')

[Errno 2] No such file or directory: '/content/drive/MyDrive/ML_FSP'
/content


FileNotFoundError: ignored

In [ ]:
songs.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


Because of the dataset being so big, we are going to resample only 5000 random songs.

In [ ]:
songs = songs.sample(n=5000).drop('link', axis=1).reset_index(drop=True)

We can notice also the presence of `\n` in the text, so we are going to remove it.

In [ ]:
songs['text'] = songs['text'].str.replace(r'\n', '')

<ipython-input-16-215fe46be90d>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  songs['text'] = songs['text'].str.replace(r'\n', '')


We use TF-IDF vectorizer that calculates the TF-IDF score for each song lyric, word-by-word.

In [ ]:
tfidf = TfidfVectorizer(analyzer='word', stop_words='english')

In [ ]:
lyrics_matrix = tfidf.fit_transform(songs['text'])

We now need to calculate the similarity of one lyric to another. We are going to use **cosine similarity**.


In [ ]:
cosine_similarities = cosine_similarity(lyrics_matrix)

Once we get the similarities, we'll store in a dictionary the names of the 50  most similar songs for each song in our dataset.

In [ ]:
similarities = {}

In [ ]:
for i in range(len(cosine_similarities)):
    # Now we'll sort each element in cosine_similarities and get the indexes of the songs.
    similar_indices = cosine_similarities[i].argsort()[:-50:-1]
    # After that, we'll store in similarities each name of the 50 most similar songs.
    # Except the first one that is the same song.
    similarities[songs['song'].iloc[i]] = [(cosine_similarities[i][x], songs['song'][x], songs['artist'][x]) for x in similar_indices][1:]

In [ ]:
class ContentBasedRecommender:
    def __init__(self, matrix):
        self.matrix_similar = matrix

    def _print_message(self, song, recom_song):
        rec_items = len(recom_song)

        print(f'The {rec_items} recommended songs for {song} are:')
        for i in range(rec_items):
            print(f"Number {i+1}:")
            print(f"{recom_song[i][1]} by {recom_song[i][2]} with {round(recom_song[i][0], 3)} similarity score")
            print("--------------------")

    def recommend(self, recommendation):
        # Get song to find recommendations for
        song = recommendation['song']
        # Get number of songs to recommend
        number_songs = recommendation['number_songs']
        # Get the number of songs most similars from matrix similarities
        recom_song = self.matrix_similar[song][:number_songs]
        # print each item
        self._print_message(song=song, recom_song=recom_song)

Now, instantiate class

In [ ]:
recommendation = ContentBasedRecommender(similarities)

Then, we are ready to pick a song from the dataset and make a recommendation.

In [ ]:
song_recommendation = {
    "song": songs['song'].iloc[10],
    "number_songs": 4
}

In [ ]:
recommendation.recommend(song_recommendation)

The 4 recommended songs for Make Sure You're Sure are:
Number 1:
Sure Feels Like Love by Demi Lovato with 0.335 similarity score
--------------------
Number 2:
It Sure Was by Kris Kristofferson with 0.273 similarity score
--------------------
Number 3:
If I Ever Fall In Love by Backstreet Boys with 0.228 similarity score
--------------------
Number 4:
Sure Thing Falling by Yellowcard with 0.206 similarity score
--------------------


And we can pick another random song and recommend again:

In [ ]:
song_recommendation2 = {
    "song": songs['song'].iloc[13],
    "number_songs": 4
}

In [ ]:
recommendation.recommend(song_recommendation2)

The 4 recommended songs for Can't Always Get What You Want are:
Number 1:
Let Me by Santana with 0.525 similarity score
--------------------
Number 2:
Want To Be With You by Moody Blues with 0.447 similarity score
--------------------
Number 3:
I Want Your Love by Lady Gaga with 0.438 similarity score
--------------------
Number 4:
I Need A Man by Gloria Estefan with 0.4 similarity score
--------------------
